In [1]:
%gui asyncio
import asyncio
import threading
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import time
import re
import serial
import sys
import glob
import serial
import numpy as np

In [2]:
#https://stackoverflow.com/questions/12090503/listing-available-com-ports-with-python
def get_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [3]:
def list_ports():
    print('Available serial ports: ')
    ports=get_ports()
    for port in ports:
        print(ports.index(port), end=':')
        print(port)
def try_connect(portindex):
    myport = None
    try:
        myport = serial.Serial(
            port=get_ports()[portindex],
            baudrate=250000,
            timeout=0,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS
        )
        print('Sucessfully connected to port '+myport.port)
        return myport
    except (OSError, serial.SerialException):
        print('ERROR: ', end='')
        if myport.is_open:
            print('Port already open')
        elif myport.port not in portlist:
            print('Port '+myport.port+' not found')
        else:
            print('Could not connect to port')
        pass

In [4]:
def read_serial():
    return myport.readline().decode().rstrip()

def contains_ok(lines):
    if 'ok' in lines:
        return True
    return False

def handle_input(sender):
    userinput=sender.value
    sender.value=''
    if userinput in macros:
        terminalout.append_display_data("MACRO: "+userinput)
        macros[userinput]()
    else:
        send_gcode(userinput)
def send_gcode(gcode, wait=True):
    if wait:
        userin.disabled=True
        userin.placeholder='Waiting for OK'
    raw = gcode.upper()
    machineout.append_display_data('SEND: '+raw)
    sInput = (raw + lineTerminator) #append terminator string
    bInput = sInput.encode() #convert string to bytes
    myport.write(bInput) #send the command

def current_milli_time():
    return round(time.time() * 1000)

def is_m114_feedback(printeroutput):
    if(len(re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})", printeroutput))==3):
        return True
    return False

def parse_coords(printeroutput):
    return list(map(float, re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})",printeroutput)))

In [5]:
## String constants ##
commandPrompt='>>: '
lineTerminator='\r\n'

## MACRO DEFINITIONS ##
def show_help():
    """Shows this help"""
    terminalout.append_display_data('Available macros:')
    for macro in macros.items():
        terminalout.append_display_data(macro[0]+':'+macro[1].__doc__)
def probe_point():
    """returns a tuple containing the probed coordinate"""
    
def home():
    """Auto Home XYZ"""
    send_gcode('G28')
    return
def deploy():
    """Deploy the probe"""
    send_gcode('M401')
    return
def stow():
    """Stow the probe"""
    send_gcode('M402')
    return
def fan_on():
    """Set case fan speed to MAX"""
    send_gcode('M106 P0 S255')
    return
def fan_off():
    """Disable all fans"""
    send_gcode('M107')
    return
def get_pos():
    """Report the current tool position"""
    send_gcode('M114')
    return
def disable_motors():
    """Disables all the motors"""
    send_gcode('M18')
    return

## MACRO DICTIONARY
macros = {
    'help':show_help,
    'probe':probe_point,
    'home':home,
    'deploy':deploy,
    'stow':stow,
    'fanon':fan_on,
    'fanoff':fan_off,
    'pos':get_pos,
    'moff':disable_motors
}


In [6]:
machineout = widgets.Output(
    layout={
            'width':'50%',
            'height':'500px',
            'border': '1px solid white',
            'overflow':'scroll',
           }
)
terminalout = widgets.Output(
    layout={
            'width':'50%',
            'height':'500px',
            'border': '1px solid white',
            'overflow':'scroll',
           }
)
userin = widgets.Text(
    layout={
            'width':'100%',
            'border': '1px solid white'
           },
    value='',
    placeholder='Type a command here',
    disabled=False
)
userin.on_submit(handle_input)
termgroup = HBox([machineout, terminalout])
controlgroup = VBox([termgroup, userin])

In [7]:
#list_ports()

In [8]:
myport=try_connect(1)

Sucessfully connected to port COM5


In [11]:
def _update():
    while True:
        serialdata = read_serial()
        if serialdata != '':
            if is_m114_feedback(serialdata):
                currentposition = parse_coords(serialdata)
                terminalout.append_display_data(currentposition)
            machineout.append_display_data(serialdata)
        if contains_ok(serialdata):
            userin.placeholder='Type a command here'
            userin.disabled=False
        time.sleep(0.01)

In [10]:
currentposition=[0,0,0]
myport.reset_input_buffer()
machineout.clear_output()
terminalout.clear_output()
t_update = threading.Thread(target=_update, args=())
display(controlgroup)
t_update.start()
print('Enter your commands in the box above. Press <ENTER> to send.')
print('Type "help" to see available shorcuts')
#x.join()

Enter your commands in the box above. Press <ENTER> to send.
Type "help" to see available shorcuts
